In [ ]:
# tensorflow version
import tensorflow
print('tensorflow: %s' % tensorflow.__version__)
# keras version
import tensorflow.keras
print('keras: %s' % tensorflow.keras.__version__)

# We create the CV model and extract features from our images

In [ ]:
from os import listdir
from pickle import dump
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB7

# extract features from each photo in the directory
def extract_features(directory):
	# load the model without predicitions layer
	model = EfficientNetB7(include_top=False, input_shape=(600, 600, 3), pooling="max")
	# summarize
	print(model.summary())
	# extract features from each photo
	features = dict()
	for name in listdir(directory):
		# load an image from file
		filename = directory + '/' + name
		image = load_img(filename, target_size=(600, 600))
		# convert the image pixels to a numpy array
		image = img_to_array(image)
		# reshape data for the model
		image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
		# prepare the image for the EfficientNet model
		image = preprocess_input(image)
		# get features
		feature = model.predict(image, verbose=2)
		# get image id
		image_id = name.split('.')[0]
		# store feature
		features[image_id] = feature
		print('>%s' % name)
	return features

# extract featflickr/Flicker8k_Datasetges
directory = 'png'
features = extract_features(directory)
print('Extracted Features: %d' % len(features))
# save to file
dump(features, open('webpagegen_10k_b7_features.pkl', 'wb'))

# We extract captions for the images and create a cleaned vocabulary + answer key

In [ ]:
import string
from os import listdir

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# transform gui dir into descriptions doc
def create_descriptions(gui_dir):
    descriptions = []
    gui_filenames = listdir(gui_dir)

    for gui_filename in gui_filenames:
        gui_id = gui_filename.split('.')[0]
        gui_contents = load_doc(gui_dir + "/" + gui_filename)
		# TEMPORARY FIX FOR DATASET MALFORMAT
        gui_contents = gui_contents.replace("body{", "body {")
        gui_contents = gui_contents.replace("table{", "table {")
		# END TEMPORARY FIX
        gui_contents = gui_contents.replace(",", " ,")
        gui_contents = ' '.join(gui_contents.split())
        descriptions.append(gui_id + ' ' + gui_contents)
    
    data = '\n'.join(descriptions)
    file = open('descriptions.txt', 'w')
    file.write(data)
    file.close()

# extract descriptions for images
def load_descriptions(doc):
	mapping = dict()
	# process lines
	for line in doc.split('\n'):
		# split line by white space
		tokens = line.split()
		if len(line) < 2:
			continue
		# take the first token as the image id, the rest as the description
		image_id, image_desc = tokens[0], tokens[1:]
		# remove filename from image id
		image_id = image_id.split('.')[0]
		# convert description tokens back to string
		image_desc = ' '.join(image_desc)
		# create the list if needed
		if image_id not in mapping:
			mapping[image_id] = list()
		# store description
		mapping[image_id].append(image_desc)
	return mapping

# convert the loaded descriptions into a vocabulary of words
def to_vocabulary(descriptions):
	# build a list of all description strings
	all_desc = set()
	for key in descriptions.keys():
		[all_desc.update(d.split()) for d in descriptions[key]]
	return all_desc

# transform gui dir to descriptions
create_descriptions('gui')
# verify descriptions
filename = 'descriptions.txt'
doc = load_doc(filename)
descriptions = load_descriptions(doc)
print('Loaded: %d ' % len(descriptions))
# summarize vocabulary
vocabulary = to_vocabulary(descriptions)
print('Vocabulary Size: %d' % len(vocabulary))

# Load the data and train

In [1]:
import tensorflow
from numpy import array
from pickle import load
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import add
from tensorflow.keras.callbacks import ModelCheckpoint

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split()[0]
		dataset.append(identifier)
	return dataset

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        # skip images not in the set
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            # store
            descriptions[image_id].append(desc)
    return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, descriptions, photos, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each image identifier
	for key, desc_list in descriptions.items():
		# walk through each description for the image
		for desc in desc_list:
			# encode the sequence
			seq = tokenizer.texts_to_sequences([desc])[0]
			# split one sequence into multiple X,y pairs
			for i in range(1, len(seq)):
				# split into input and output pair
				in_seq, out_seq = seq[:i], seq[i]
				# pad input sequence
				in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
				# encode output sequence
				out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
				# store
				X1.append(photos[key][0])
				X2.append(in_seq)
				y.append(out_seq)
	return array(X1), array(X2), array(y)

# define the captioning model
def define_model(vocab_size, max_length):
	# feature extractor model
	inputs1 = Input(shape=(2560,))
	fe1 = Dropout(0.5)(inputs1)
	fe2 = Dense(256, activation='relu')(fe1)
	fe3 = tensorflow.keras.layers.RepeatVector(max_length)(fe2) # EXPERIMENTAL
	# sequence model
	inputs2 = Input(shape=(max_length,))
	se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
	se2 = Dropout(0.5)(se1)
	se3 = LSTM(128, return_sequences=True)(se2)
	se4 = LSTM(128, return_sequences=True)(se3) # EXPERIMENTAL
	# # decoder model ORIGINAL
	# decoder1 = add([fe2, se3])
	# decoder2 = Dense(256, activation='relu')(decoder1)
	# outputs = Dense(vocab_size, activation='softmax')(decoder2)
	# decoder model EXPERIMENTAL
	decoder1 = tensorflow.keras.layers.Concatenate()([fe3, se4])
	decoder2 = LSTM(512, return_sequences=True)(decoder1)
	decoder3 = LSTM(512, return_sequences=False)(decoder2)
	outputs = Dense(vocab_size, activation='softmax')(decoder3)
	# tie it together [image, seq] [word]
	model = Model(inputs=[inputs1, inputs2], outputs=outputs)
	model.compile(loss='categorical_crossentropy', optimizer='adam')
	# summarize model
	print(model.summary())
	plot_model(model, to_file='model.png', show_shapes=True)
	return model

# train dataset

# load training dataset (8K)
descriptions = 'descriptions.txt'
features = 'webpagegen_10k_b7_features.pkl'
dataset = load_set(descriptions)
print('Dataset: %d' % len(dataset))

# split into train and test sets
train = set(dataset[:8000])
test = set(dataset[8000:])

# descriptions
train_descriptions = load_clean_descriptions(descriptions, train)
print('Descriptions: train=%d' % len(train_descriptions))
test_descriptions = load_clean_descriptions(descriptions, test)
print('Descriptions: test=%d' % len(test_descriptions))

# photo features
train_features = load_photo_features(features, train)
print('Photos: train=%d' % len(train_features))
test_features = load_photo_features(features, test)
print('Photos: test=%d' % len(test_features))

# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# prepare sequences
print("Encoding training data...")
X1train, X2train, ytrain = create_sequences(tokenizer, max_length, train_descriptions, train_features, vocab_size)
print("Encoding testing data...")
X1test, X2test, ytest = create_sequences(tokenizer, max_length, test_descriptions, test_features, vocab_size)
print("Encoding complete")

# define the model
model = define_model(vocab_size, max_length)
# define checkpoint callback
filepath = 'run2-801/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

Dataset: 10000
Descriptions: train=8000
Descriptions: test=2000
Photos: train=8000
Photos: test=2000
Vocabulary Size: 25
Description Length: 801
Encoding training data...
Encoding testing data...
Encoding complete
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 801)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 2560)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 801, 256)     6400        input_2[0][0]                    
______________________________________________________________________________

In [2]:
# fit model
print("Let the training commence!")
history = model.fit([X1train, X2train], ytrain, epochs=20, verbose=1, callbacks=[checkpoint], validation_data=([X1test, X2test], ytest), batch_size=128)

Let the training commence!
Epoch 1/20
  74/2926 [..............................] - ETA: 8:22:25 - loss: 2.9605

KeyboardInterrupt: 

# Evaluate

In [22]:
import tensorflow
from numpy import argmax
from pickle import load
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split()[0]
		dataset.append(identifier)
	return dataset

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        # skip images not in the set
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            # store
            descriptions[image_id].append(desc)
    return descriptions

# load photo features
def load_photo_features(filename, dataset):
	# load all features
	all_features = load(open(filename, 'rb'))
	# filter features
	features = {k: all_features[k] for k in dataset}
	return features

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the length of the description with the most words
def max_length(descriptions):
	lines = to_lines(descriptions)
	return max(len(d.split()) for d in lines)

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
    # seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
        # predict next word
		yhat = model.predict([photo,sequence], verbose=0)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# evaluate the skill of the model
def evaluate_model(model, descriptions, photos, tokenizer, max_length):
	actual, predicted = list(), list()
	# step over the whole set
	for key, desc_list in descriptions.items():
		# generate description
		yhat = generate_desc(model, tokenizer, photos[key], max_length)
		# store actual and predicted
		references = [d.split() for d in desc_list]
		actual.append(references)
		predicted.append(yhat.split())
	# calculate BLEU score
	print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
	print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
	print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
	print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))


# train dataset

# load training dataset (8K)
descriptions = 'descriptions.txt'
features = 'webpagegen_10k_b7_features.pkl'
dataset = load_set(descriptions)
print('Dataset: %d' % len(dataset))

# split into train and test sets
train = set(dataset[:8000])
test = set(dataset[8000:])

# descriptions
train_descriptions = load_clean_descriptions(descriptions, train)
print('Descriptions: train=%d' % len(train_descriptions))
test_descriptions = load_clean_descriptions(descriptions, test)
print('Descriptions: test=%d' % len(test_descriptions))

# photo features
train_features = load_photo_features(features, train)
print('Photos: train=%d' % len(train_features))
test_features = load_photo_features(features, test)
print('Photos: test=%d' % len(test_features))

# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

# determine the maximum sequence length
max_length = max_length(train_descriptions)
print('Description Length: %d' % max_length)

# load the model
filename = 'run1/model-ep001-loss1.495-val_loss1.197.h5'
model = load_model(filename)

# evaluate model
evaluate_model(model, test_descriptions, test_features, tokenizer, 772)

Dataset: 10000
Descriptions: train=8000
Descriptions: test=2000
Photos: train=8000
Photos: test=2000
Vocabulary Size: 25
Description Length: 801


OSError: SavedModel file does not exist at: run1/model-ep001-loss1.495-val_loss1.197.h5/{saved_model.pbtxt|saved_model.pb}

# Predict

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer
from pickle import dump

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load a pre-defined list of photo identifiers
def load_set(filename):
	doc = load_doc(filename)
	dataset = list()
	# process line by line
	for line in doc.split('\n'):
		# skip empty lines
		if len(line) < 1:
			continue
		# get the image identifier
		identifier = line.split()[0]
		dataset.append(identifier)
	return dataset

# load clean descriptions into memory
def load_clean_descriptions(filename, dataset):
    # load document
    doc = load_doc(filename)
    descriptions = dict()
    for line in doc.split('\n'):
        # split line by white space
        tokens = line.split()
        # split id from description
        image_id, image_desc = tokens[0], tokens[1:]
        # skip images not in the set
        if image_id in dataset:
            # create list
            if image_id not in descriptions:
                descriptions[image_id] = list()
            # wrap description in tokens
            desc = 'startseq ' + ' '.join(image_desc) + ' endseq'
            # store
            descriptions[image_id].append(desc)
    return descriptions

# covert a dictionary of clean descriptions to a list of descriptions
def to_lines(descriptions):
	all_desc = list()
	for key in descriptions.keys():
		[all_desc.append(d) for d in descriptions[key]]
	return all_desc

# fit a tokenizer given caption descriptions
def create_tokenizer(descriptions):
	lines = to_lines(descriptions)
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# load training dataset (8K)
descriptions = 'descriptions.txt'
dataset = load_set(descriptions)
print('Dataset: %d' % len(dataset))

# split into train set
train = set(dataset[:8000])

# descriptions
train_descriptions = load_clean_descriptions(descriptions, train)
print('Descriptions: train=%d' % len(train_descriptions))

# prepare tokenizer
tokenizer = create_tokenizer(train_descriptions)

# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

Dataset: 10000
Descriptions: train=8000


In [27]:
from pickle import load
from numpy import argmax
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.applications import EfficientNetB7

# extract features from each photo in the directory
def extract_features(filename):
	# load the model
	model = EfficientNetB7(include_top=False, input_shape=(600, 600, 3), pooling="max")
	# load the photo
	image = load_img(filename, target_size=(600, 600))
	# convert the image pixels to a numpy array
	image = img_to_array(image)
	# reshape data for the model
	image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
	# prepare the image for the B7 model
	image = preprocess_input(image)
	# get features
	feature = model.predict(image, verbose=1)
	return feature

# map an integer to a word
def word_for_id(integer, tokenizer):
	for word, index in tokenizer.word_index.items():
		if index == integer:
			return word
	return None

# generate a description for an image
def generate_desc(model, tokenizer, photo, max_length):
	# seed the generation process
	in_text = 'startseq'
	# iterate over the whole length of the sequence
	for i in range(max_length):
		# integer encode input sequence
		sequence = tokenizer.texts_to_sequences([in_text])[0]
		# pad input
		sequence = pad_sequences([sequence], maxlen=max_length)
		# predict next word
		yhat = model.predict([photo,sequence], verbose=1)
		# convert probability to integer
		yhat = argmax(yhat)
		# map integer to word
		word = word_for_id(yhat, tokenizer)
		# stop if we cannot map the word
		if word is None:
			break
		# append as input for generating the next word
		in_text += ' ' + word
		# stop if we predict the end of the sequence
		if word == 'endseq':
			break
	return in_text

# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 772
# load the model
model = load_model('run1-772/model-ep013-loss1.159-val_loss1.146.h5')
# load and prepare the photograph
photo = extract_features('example.png')
# generate description
description = generate_desc(model, tokenizer, photo, max_length)
print(description)

1/1 [==============================] - 0s 316ms/step
startseq body heading form link link link link submit navbar btn dropdown checkbox radio radio radio radio radio radio radio radio search btn heading form link link link link submit navbar btn heading form link link link link submit navbar btn heading form link link link link submit navbar btn heading form link link link link submit navbar btn heading form link link link link submit navbar btn heading form link link link link submit navbar btn heading form link link link link submit navbar btn dropdown radio radio radio radio radio radio radio radio search btn heading form link link link link submit navbar btn logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 logo 3 heading form link link link link submit navbar btn endseq
